# 3D Mesh Analysis
This notebook contains methods for 3D mesh analysis and visualization. It is dependent on the __plotly__ package for data and mesh visualization, and on common scipy packages like __numpy__ for data processing. All geometric features are computed natively without other external dependencies to ensure correctness.


In [1]:
#Setup and check required packages
import plotly
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
print("Plotly: ",plotly.__version__)
import numpy as np
print("NumPy: ",np.__version__)
import matplotlib as mplt
import matplotlib.pyplot as plt
print("Matplotlib: ", mplt.__version__)

import time

Plotly:  3.10.0
NumPy:  1.16.3
Matplotlib:  3.0.3


https://plot.ly/python/

## Action Items:
* ~~Mesh extraction~~
* ~~Mesh visualization~~
* ~~Normal vector calculation, visualization~~
* ~~Mean curvature calculation, visualization~~
* ~~Corrected curvature calculation, zero handling~~
* ~~Directional curvature components~~
* ~~Normal vector histogram~~
* ~~Directional curvature histograms~~
* ~~Ear tip and nose geometric identification~~
* ~~Improved geometric identification robustness~~
* ~~Region selection based on geometric landmarks~~
* ~~Region selection based on normal vector~~
* ~~Bulk processing~~
* Old metrics versus improved metrics
* ~~Metric analysis between groups~~
* Improve by preprocessing head alignment and position

* ~~Object-oriented~~
* ~~Surface area metrics~~
* ~~Debug landmark creation~~
* ~~Results and group results~~

* ~~Split patient load from region/result update~~
* ~~Multiple regions - front half vs back half of cranium~~
* ~~Result - per-vertex ratio of x to total curvature~~
* Debug directional curvature magnitude
* T test
* Analysis - separation of pre vs post for each metric - T test all
* Analysis of best metric between groups
* Plot metrics versus post-op month to see age correlation

Extracted data features:

    vertices
    faces
    minX/maxX/rangeX
    maxrange
    vertexNeighbors
    faceNeighbors
    faceAreas
    faceNormals
    vertexNormals
    vertexElevation
    vertexAzimuth
    vertexCurvatures
    vertexDirectionalCurvatures
    leftEarIndex, rightEarIndex, nasionIndex, noseIndex

BEGIN  - Classes and Functions

In [2]:
class Mesh:
    def __init__(self, vertices, faces):
        self.vertices = vertices
        self.faces = faces
        
        timeDebug = False
        
        start_time = time.time()
        prev_time = time.time()
        
        #Check that all faces are triangular
        if not sum([1 if len(face)>3 else 0 for face in faces])==0: print('All Triangular: False')
        #Flip X sign
        self.vertices[:,0]*=-1.0
    
        #Swap Y and Z to make movement better in plotly
        tempX = np.copy(self.vertices[:,0])
        tempY = np.copy(self.vertices[:,1])
        tempZ = np.copy(self.vertices[:,2])
        self.vertices[:,1]=tempZ
        self.vertices[:,2]=tempY
    
        #Get max/min ranges
        self.maxX=np.max(self.vertices[:,0])
        self.maxY=np.max(self.vertices[:,1])
        self.maxZ=np.max(self.vertices[:,2])
        self.minX=np.min(self.vertices[:,0])
        self.minY=np.min(self.vertices[:,1])
        self.minZ=np.min(self.vertices[:,2])
        self.rangeX=self.maxX-self.minX
        self.rangeY=self.maxY-self.minY
        self.rangeZ=self.maxZ-self.minZ
        self.maxRange=max(self.rangeX,self.rangeY,self.rangeZ)
        #print("MaxX:",maxX," MaxY:",maxY," MaxZ:",maxZ)
        #print("MinX:",minX," MinY:",minY," MinZ:",minZ)
        
        range_time = time.time()-prev_time
        prev_time = time.time()

        
        #Construct list of vertex neighbors for each vertex
        self.vertexNeighbors=[]
        for v_ind in range(0,self.vertices.shape[0]):
            mask=np.any(np.isin(faces,v_ind),1)
            neighbors=np.unique(faces[mask])
            neighbors=np.delete(neighbors,np.argwhere(neighbors==v_ind))
            self.vertexNeighbors.append(neighbors)
            
        vertNeighbor_time = time.time()-prev_time
        prev_time = time.time()

            
        #Construct list of face neighbors for each vertex
        self.faceNeighbors=[]
        for v_ind in range(0,self.vertices.shape[0]):
            neighbors=np.nonzero(np.any(np.isin(faces,v_ind),1))
            self.faceNeighbors.append(neighbors)

        faceNeighbor_time = time.time()-prev_time
        prev_time = time.time()
            
        #Calculate normal and area of each face
        vA=self.vertices[faces[:,0]]
        vB=self.vertices[faces[:,1]]
        vC=self.vertices[faces[:,2]]
        eU=vB-vA
        eV=vC-vA
        dirVecs=np.cross(eU,eV)
        self.faceAreas=np.linalg.norm(dirVecs,ord=2,axis=1,keepdims=True)
        self.faceNormals=dirVecs/self.faceAreas
        
        #Calculate vertex normals using face normals
        vert_norms_list=[]
        for v_ind in range(0,vertices.shape[0]):
            scaledNorms=self.faceAreas[self.faceNeighbors[v_ind]]*self.faceNormals[self.faceNeighbors[v_ind]]
            weightedSum=np.sum(scaledNorms,axis=0)
            weight=np.linalg.norm(weightedSum)
            if(weight==0):
                vert_norms_list.append(np.zeros(3))
            else:
                vert_norms_list.append(weightedSum/weight)
        self.vertexNormals=np.array(vert_norms_list)

        normalArea_time = time.time()-prev_time
        prev_time = time.time()
        
        #Calculate elevation and azimuth at each vertex
        smallVal=0.00000000001
        horiz=np.sqrt(self.vertexNormals[:,0]**2.0+self.vertexNormals[:,1]**2.0)+smallVal
        vert=self.vertexNormals[:,2]-smallVal
        self.vertexElevation=np.arctan(vert/horiz)*180.0/np.pi

        self.vertexAzimuth=np.arctan((self.vertexNormals[:,1])/(self.vertexNormals[:,0]+smallVal))-np.pi/2
        self.vertexAzimuth[self.vertexNormals[:,0]<0]+=np.pi
        self.vertexAzimuth*=(-1.0*180.0/np.pi)

        angle_time = time.time()-prev_time
        prev_time = time.time()
        
        # Calculate curvature at each vertex using nearest neighbors
        # https://computergraphics.stackexchange.com/questions/1718/what-is-the-simplest-way-to-compute-principal-curvature-for-a-mesh-triangle

        vert_curv_list=[]
        vert_dir_curv_list=[]

        nonZeroVal=0.00003

        for v_ind in range(0, self.vertices.shape[0]):
            posDif= self.vertices[v_ind]- self.vertices[ self.vertexNeighbors[v_ind]]
            posWeight=np.linalg.norm(posDif,ord=2,axis=1,keepdims=True)
            normDif= self.vertexNormals[v_ind]- self.vertexNormals[ self.vertexNeighbors[v_ind]]

            #curves=(normDif*posDif)/(posWeight*posWeight)
            curves=np.sum(normDif*posDif, axis=1)
            #Normalize by posWeight squared
            curves=curves/(np.square(posWeight)).T
            #Geometric mean of each edge
            curve=np.prod(np.abs(curves))**(1.0/curves.size)
            # Zero curve breaks log analysis, occurs for points with only one/two faces
            if(curve==0.0):
                curve=nonZeroVal
            vert_curv_list.append(curve)

            #Breakdown curvature dimenesion by normalized directional component of each edge
            dirCurves=np.abs((curves.T*posDif)/posWeight)    
            dirCurve=np.prod(np.abs(dirCurves),axis=0)**(1.0/dirCurves.shape[0])
            # Zero curve breaks log analysis
            if(not np.all(dirCurve)):
                dirCurve=[nonZeroVal/3.0,nonZeroVal/3.0,nonZeroVal/3.0]
            vert_dir_curv_list.append(dirCurve)

        self.vertexCurvatures=np.array(vert_curv_list)
        self.vertexDirectionalCurvatures=np.array(vert_dir_curv_list)
        
        curvature_time = time.time()-prev_time
        prev_time = time.time()
            
        self.findLandmarks()
    
        landmark_time = time.time()-prev_time
        prev_time = time.time()
            
        total_time = prev_time - start_time
        
        if(timeDebug):
            print("Total:",total_time,", Range:",range_time/total_time,", VertNeighbor:",vertNeighbor_time/total_time,", FaceNeighbor:",faceNeighbor_time/total_time, ", NormalArea:",normalArea_time/total_time,", Angle:",angle_time/total_time,", Curvature:",curvature_time/total_time,", Landmark:",landmark_time/total_time)
    
    def findLandmarks(self):
        #vertexColors=np.ones_like(vertices)

        #start at Max X vertex, then search up Z
        currInd=np.argmax(self.vertices[:,0])
        currZ=self.vertices[currInd,2]
        #vertexColors[currInd,:]=[0.0,1.0,1.0]
        #vertexColors[vertexNeighbors[currInd],:]*=[0.0,0.0,1.0]

        #print("Start Right Ear:",currInd,", Z:",currZ)
        # DYLAN TODO Turned off for now, often going to top of head
        #while(True):
        #    neighborInd=self.vertexNeighbors[currInd]
        #    neighborPos=self.vertices[neighborInd]
        #    if(np.max(neighborPos[:,2])>currZ):
        #        currInd=neighborInd[np.argmax(neighborPos[:,2])]
        #        currZ=self.vertices[currInd,2]
        #        #vertexColors[currInd,:]*=[0.0,1.0,0.0]
        #    else:
        #        break
        
        #print("Final Right Ear:",currInd,", Z:",currZ)
        self.rightEarIndex=currInd
        #vertexColors[currInd,:]=[1.0,1.0,0.0]
        #vertexColors[vertexNeighbors[currInd],:]*=[1.0,0.0,0.0]

        #start at Min X vertex, then search up Z
        currInd=np.argmin(self.vertices[:,0])
        currZ=self.vertices[currInd,2]
        #vertexColors[currInd,:]=[0.0,1.0,1.0]
        #vertexColors[vertexNeighbors[currInd],:]*=[0.0,0.0,1.0]
        #print("Start Left Ear:",currInd,", Z:",currZ)
        # DYLAN TODO Turned off for now, often going to top of head
        #while(True):
        #    neighborInd=self.vertexNeighbors[currInd]
        #    neighborPos=self.vertices[neighborInd]
        #    if(np.max(neighborPos[:,2])>currZ):
        #        currInd=neighborInd[np.argmax(neighborPos[:,2])]
        #        currZ=self.vertices[currInd,2]
        #        #vertexColors[currInd,:]*=[0.0,1.0,0.0]
        #    else:
        #        break
        #print("Final Left Ear:",currInd,", Z:",currZ)
        self.leftEarIndex=currInd
        #vertexColors[currInd,:]=[1.0,1.0,0.0]
        #vertexColors[vertexNeighbors[currInd],:]*=[1.0,0.0,0.0]

        #start at Max Y vertex less than 0 Z, always move up Z, always move down Y, minimize X distance
        tempVerts = np.copy(self.vertices)
        tempVerts[tempVerts[:,2]>0,1]*=0.0
        currInd=np.argmax(tempVerts[:,1])
        [currX,currY,currZ]=self.vertices[currInd]
        #vertexColors[currInd,:]=[0.0,1.0,1.0]
        #vertexColors[vertexNeighbors[currInd],:]*=[0.0,0.0,1.0]

        #print("Start Nose:",currInd,", X:",currX,", Y:",currY,", Z:",currZ)
        self.noseIndex=currInd
        while(True):
            neighborInd=self.vertexNeighbors[currInd]
            neighborPos=self.vertices[neighborInd]
            #Only larger Z values
            #Z-band robustness: base on average neighbor Z delta
            neighborZDriftAverage=np.average(np.abs(neighborPos[:,2]-currZ))
            neighborInd=neighborInd[neighborPos[:,2]>(currZ+(0.75*neighborZDriftAverage))]
            neighborPos=self.vertices[neighborInd]
            #Only smaller Y values
            neighborInd=neighborInd[neighborPos[:,1]<currY]
            neighborPos=self.vertices[neighborInd]
            if(neighborPos.size==0):
                break
            #Decision: of remaining neighbors, minimize X drift
            neighborXDrift=np.abs(neighborPos[:,0]-currX)
            currInd=neighborInd[np.argmin(neighborXDrift)]
            [currX,currY,currZ]=self.vertices[currInd]
            #vertexColors[currInd,:]*=[0.0,1.0,0.0]
        #print("Final Nasion:",currInd,", X:",currX,", Y:",currY,", Z:",currZ)
        self.nasionIndex=currInd
        #vertexColors[currInd,:]=[1.0,1.0,0.0]
        #vertexColors[vertexNeighbors[currInd],:]*=[1.0,0.0,0.0]

        #From nasion, continue to brow ridge (TODO what is formal name?)
        while(True):
            neighborInd=self.vertexNeighbors[currInd]
            neighborPos=self.vertices[neighborInd]
            #Only larger Z values
            #Z-band robustness: base on average neighbor Z delta
            neighborZDriftAverage=np.average(np.abs(neighborPos[:,2]-currZ))
            neighborInd=neighborInd[neighborPos[:,2]>(currZ+(0.75*neighborZDriftAverage))]
            neighborPos=self.vertices[neighborInd]
            #Only greater Y values
            neighborInd=neighborInd[neighborPos[:,1]>currY]
            neighborPos=self.vertices[neighborInd]
            if(neighborPos.size==0):
                break
            #Decision: of remaining neighbors, minimize X drift
            #Test, minimize Y drift instead
            neighborXDrift=np.abs(neighborPos[:,0]-currX)
            neighborYDrift=np.abs(neighborPos[:,1]-currY)
            #currInd=neighborInd[np.argmin(neighborXDrift)]
            #currInd=neighborInd[np.argmin(neighborYDrift)]
            currInd=neighborInd[np.argmin(neighborYDrift+neighborXDrift)]
            [currX,currY,currZ]=self.vertices[currInd]
            #vertexColors[currInd,:]*=[0.0,1.0,0.0]
        #print("Final Brow Ridge:",currInd,", X:",currX,", Y:",currY,", Z:",currZ)
        self.browRidgeIndex=currInd
        #vertexColors[currInd,:]=[1.0,1.0,0.0]
        #vertexColors[vertexNeighbors[currInd],:]*=[1.0,0.0,0.0]
        #plot_mesh(vertices,faces,vertexColors,"Ear Points: Blue=[Min|Max X] Red=[Local Max Z]<br>Nose Point: Blue=[Max Y] Red=[Min X Drift, Reduce Y, Increase Z (Banded)]")


In [3]:
#Function to capture NP arrays of vertices and faces

def obj_data_to_Mesh(odata):
    # odata is the string read from an obj file
    vertices = []
    faces = []
    lines = odata.splitlines()   
   
    for line in lines:
        slist = line.split()
        if slist:
            if slist[0] == 'v':
                vertex = list(map(float, slist[1:]))
                vertices.append(vertex)
            elif slist[0] == 'f':
                face = []
                for k in range(1, len(slist)):
                    face.append([int(s) for s in slist[k].replace('//','/').split('/')])
                
                if len(face) > 3: # triangulate the n-polyonal face, n>3
                    faces.extend([[face[0][0]-1, face[k][0]-1, face[k+1][0]-1] for k in range(1, len(face)-1)])
                else:    
                    faces.append([face[j][0]-1 for j in range(len(face))])
            else: pass
    
    
    return Mesh(vertices=np.array(vertices), faces=np.array(faces))    

In [4]:
init_notebook_mode(connected=True)

def plot_hist(dataset,dataname):
    plt.hist(dataset)
    datatitle=dataname+" Histogram"
    plt.title(datatitle)
    plt.xlabel(dataname)
    plt.ylabel("Frequency")
    fig = plt.gcf()
    plotly_fig=tls.mpl_to_plotly(fig)
    iplot(plotly_fig)

In [5]:
#Display 3D Mesh using plotly
init_notebook_mode(connected=True)

def plot_mesh_custom(mesh,vert_colors,title="3D Mesh"):

    x,y,z=mesh.vertices.T
    I,J,K=mesh.faces.T
    mesh3d=dict(type='mesh3d',
            x=x,
            y=y,
            z=z,
            vertexcolor=vert_colors,
            i=I,
            j=J,
            k=K,
            name='',
            showscale=False
        )
    mesh3d.update(lighting=dict(ambient=0.6,
                             diffuse=0.4,
                             fresnel=0.2,
                             specular=0.0,
                             roughness=0.9),
               lightposition=dict(x=200,
                                 y=200,
                                 z=200))
    layout=dict(title=title,
               font=dict(size=14, color="black"),
               width=750,
               height=750,
               scene=dict(xaxis=dict(visible=True),
                         yaxis=dict(visible=True),
                         zaxis=dict(visible=True),
                         aspectratio=dict(x=mesh.rangeX/mesh.maxRange,
                                         y=mesh.rangeY/mesh.maxRange,
                                         z=mesh.rangeZ/mesh.maxRange
                                         ),
                         camera=dict(eye=dict(x=1.10,y=1.10,z=1.10)),
                         ),
               paper_bgcolor='rgb(235,235,235)',
               margin=dict(t=50)
               )

    fig=go.Figure(data=[mesh3d], layout=layout)
    iplot(fig)

In [6]:
def plot_mesh_colorscale(mesh,intensities,title="3D Mesh",intensityTitle="Intensity"):

    x,y,z=mesh.vertices.T
    I,J,K=mesh.faces.T
    mesh3d=dict(type='mesh3d',
            x=x,
            y=y,
            z=z,
            colorbar=dict(title=intensityTitle),
            colorscale=[[0, 'rgb(0, 0, 255)'],
                      [1, 'rgb(255, 0, 0)']],
              #                      [0.5, 'rgb(0, 255, 0)'], 
                                      #[0.5, 'rgb(127, 127, 127)'],


            intensity=intensities,
            i=I,
            j=J,
            k=K,
            name='',
            showscale=True
        )
    mesh3d.update(lighting=dict(ambient=0.6,
                             diffuse=0.4,
                             fresnel=0.2,
                             specular=0.0,
                             roughness=0.9),
               lightposition=dict(x=200,
                                 y=200,
                                 z=200))
    layout=dict(title=title,
               font=dict(size=14, color="black"),
               width=750,
               height=750,
               scene=dict(xaxis=dict(visible=True),
                         yaxis=dict(visible=True),
                         zaxis=dict(visible=True),
                         aspectratio=dict(x=mesh.rangeX/mesh.maxRange,
                                         y=mesh.rangeY/mesh.maxRange,
                                         z=mesh.rangeZ/mesh.maxRange
                                         ),
                         camera=dict(eye=dict(x=1.10,y=1.10,z=1.10)),
                         ),
               paper_bgcolor='rgb(235,235,235)',
               margin=dict(t=50)
               )

    fig=go.Figure(data=[mesh3d], layout=layout)
    iplot(fig)

In [20]:
class Patient:
    def __init__(self,preOpMesh,postOpMesh,group,patientNumber,operationType,postOpYears):
        self.preOpMesh=preOpMesh
        self.postOpMesh=postOpMesh
        self.group = group
        self.patientNumber = patientNumber
        self.operationType = operationType
        self.postOpYears = postOpYears
        #print("debug")
        #print(len(self.preOpMesh.vertices))
        #print(len(self.postOpMesh.vertices))

In [8]:
class Result:
    def __init__(self,patient):

        
        preOpRegion=self.preFrontRegionVertices()
        self.preMeanCurv=np.mean(patient.preOpMesh.vertexCurvatures[preOpRegion])
        self.preMeanCurvX=np.mean(patient.preOpMesh.vertexDirectionalCurvatures[preOpRegion,0])
        self.preMeanCurvY=np.mean(patient.preOpMesh.vertexDirectionalCurvatures[preOpRegion,1])
        self.preMeanCurvZ=np.mean(patient.preOpMesh.vertexDirectionalCurvatures[preOpRegion,2])
        self.preRegionArea=0.0
        if(len(preOpRegion)==0):
            print("No preOpRegion for patient ",self.patient.patientNumber,"of group ",self.patient.group)
        for v in np.nditer(np.nonzero(preOpRegion)):
            self.preRegionArea+=(np.sum(patient.preOpMesh.faceAreas[patient.preOpMesh.faceNeighbors[v]]))
        self.preTotalArea = np.sum(patient.preOpMesh.faceAreas[:])
        self.preRegionAreaPercentage = self.preRegionArea / self.preTotalArea
        
        postOpRegion=self.postFrontRegionVertices()
        #print("debug2")
        #print(postOpRegion)
        #print(np.nonzero(postOpRegion))
        self.postMeanCurv=np.mean(patient.postOpMesh.vertexCurvatures[postOpRegion])
        self.postMeanCurvX=np.mean(patient.postOpMesh.vertexDirectionalCurvatures[postOpRegion,0])
        self.postMeanCurvY=np.mean(patient.postOpMesh.vertexDirectionalCurvatures[postOpRegion,1])
        self.postMeanCurvZ=np.mean(patient.postOpMesh.vertexDirectionalCurvatures[postOpRegion,2])
        self.postRegionArea=0.0
        for v in np.nditer(np.nonzero(postOpRegion)):
            self.postRegionArea+=(np.sum(patient.postOpMesh.faceAreas[patient.postOpMesh.faceNeighbors[v]]))
        self.postTotalArea = np.sum(patient.postOpMesh.faceAreas[:])
        self.postRegionAreaPercentage = self.postRegionArea / self.postTotalArea
        
        self.deltaMeanCurv = self.postMeanCurv - self.preMeanCurv
        self.deltaMeanCurvX = self.postMeanCurvX - self.preMeanCurvX
        self.deltaMeanCurvY = self.postMeanCurvY - self.preMeanCurvY
        self.deltaMeanCurvZ = self.postMeanCurvZ - self.preMeanCurvZ
        self.deltaRegionArea = self.postRegionArea - self.preRegionArea
        self.deltaRegionAreaPercentage = self.postRegionAreaPercentage - self.preRegionAreaPercentage
        self.deltaTotalArea = self.postTotalArea - self.preTotalArea

    def preFrontRegionVertices(self):
        #Cropping test based on geometric landmarks and normal vector angles (azimuth and elevation)

        #vertexColors=np.ones_like(vertices)

        #Crop mesh below the plane of the brow ridge and ear tips
        # to reduce noise from face and hair
        preMesh = self.patient.preOpMesh
        brp=preMesh.vertices[preMesh.browRidgeIndex]
        rep=preMesh.vertices[preMesh.rightEarIndex]
        lep=preMesh.vertices[preMesh.leftEarIndex]
        planeNormal=np.cross(rep-brp,lep-brp)
        abovePlane=np.sum((preMesh.vertices-brp)*planeNormal,axis=1)<-5.0

        earY=(rep[1]+lep[1])/2
        inFront=preMesh.vertices[:,1]>earY
        #Crop based on range of elevation
        #elevationCropIndidices=(np.logical_and(preMesh.vertexElevation>20,preMesh.vertexElevation<45))
        #Crop based on range of azimuth  
        #azimuthCropIndidices=(np.logical_and(preMesh.vertexAzimuth>-45,preMesh.vertexAzimuth<45))
        #Combine
        #normalCropIndicices=np.logical_and(elevationCropIndidices,azimuthCropIndidices)
        #Combine all
        #finalCropIndices=np.nonzero(np.logical_and(abovePlane,normalCropIndicices))
        #finalCropIndices=np.logical_and(abovePlane,normalCropIndicices)
        
        ##DYLAN FOR NOW USE ENTIRE PLANE, NOT JUST ANGLES
        finalCropIndices=np.logical_and(abovePlane,inFront)

        return finalCropIndices

    
    def postFrontRegionVertices(self):
        #Cropping test based on geometric landmarks and normal vector angles (azimuth and elevation)

        #Crop mesh below the plane of the brow ridge and ear tips
        # to reduce noise from face and hair
        postMesh = self.patient.postOpMesh
        brp=postMesh.vertices[postMesh.browRidgeIndex]
        rep=postMesh.vertices[postMesh.rightEarIndex]
        lep=postMesh.vertices[postMesh.leftEarIndex]
        planeNormal=np.cross(rep-brp,lep-brp)
        abovePlane=np.sum((postMesh.vertices-brp)*planeNormal,axis=1)<-5.0

        earY=(rep[1]+lep[1])/2
        inFront=postMesh.vertices[:,1]>earY

        finalCropIndices=np.logical_and(abovePlane,inFront)

        return finalCropIndices
        
    def preBackRegionVertices(self):
        #Cropping test based on geometric landmarks and normal vector angles (azimuth and elevation)

        #vertexColors=np.ones_like(vertices)

        #Crop mesh below the plane of the brow ridge and ear tips
        # to reduce noise from face and hair
        preMesh = self.patient.preOpMesh
        brp=preMesh.vertices[preMesh.browRidgeIndex]
        rep=preMesh.vertices[preMesh.rightEarIndex]
        lep=preMesh.vertices[preMesh.leftEarIndex]
        planeNormal=np.cross(rep-brp,lep-brp)
        abovePlane=np.sum((preMesh.vertices-brp)*planeNormal,axis=1)<-5.0

        earY=(rep[1]+lep[1])/2
        inBack=preMesh.vertices[:,1]<earY
        #Crop based on range of elevation
        #elevationCropIndidices=(np.logical_and(preMesh.vertexElevation>20,preMesh.vertexElevation<45))
        #Crop based on range of azimuth  
        #azimuthCropIndidices=(np.logical_and(preMesh.vertexAzimuth>-45,preMesh.vertexAzimuth<45))
        #Combine
        #normalCropIndicices=np.logical_and(elevationCropIndidices,azimuthCropIndidices)
        #Combine all
        #finalCropIndices=np.nonzero(np.logical_and(abovePlane,normalCropIndicices))
        #finalCropIndices=np.logical_and(abovePlane,normalCropIndicices)
        
        ##DYLAN FOR NOW USE ENTIRE PLANE, NOT JUST ANGLES
        finalCropIndices=np.logical_and(abovePlane,inBack)

        return finalCropIndices

    
    def postBackRegionVertices(self):
        #Cropping test based on geometric landmarks and normal vector angles (azimuth and elevation)

        #Crop mesh below the plane of the brow ridge and ear tips
        # to reduce noise from face and hair
        postMesh = self.patient.postOpMesh
        brp=postMesh.vertices[postMesh.browRidgeIndex]
        rep=postMesh.vertices[postMesh.rightEarIndex]
        lep=postMesh.vertices[postMesh.leftEarIndex]
        planeNormal=np.cross(rep-brp,lep-brp)
        abovePlane=np.sum((postMesh.vertices-brp)*planeNormal,axis=1)<-5.0

        earY=(rep[1]+lep[1])/2
        inBack=postMesh.vertices[:,1]<earY

        finalCropIndices=np.logical_and(abovePlane,inBack)

        return finalCropIndices

In [36]:
class Results:
    def __init__(self,patient,isPre=False):
        
        self.patient = patient
        self.group = patient.group
        self.patientNumber = patient.patientNumber
        self.operationType = patient.operationType
        self.postOpYears = patient.postOpYears
        
        if(isPre):
            self.mesh=self.patient.preOpMesh
        else:
            self.mesh=self.patient.postOpMesh
                
        
        self.vertexResults=dict()
        self.vertexMeans=dict()
        self.vertexSTDs=dict()
        
        self.vertexMetrics=list()
        
        
        self.globalResults=dict()
        self.globalMetrics=list()

        regionFront=self.frontRegionVertices()
        
        self.vertexResults["curvFront"]=self.mesh.vertexCurvatures[regionFront]
        self.vertexResults["curvXFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,0]
        self.vertexResults["curvYFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,1]
        self.vertexResults["curvZFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,2]
        self.vertexResults["curvXPercentFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,0]/self.mesh.vertexCurvatures[regionFront]
        self.vertexResults["curvYPercentFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,1]/self.mesh.vertexCurvatures[regionFront]
        self.vertexResults["curvZPercentFront"]=self.mesh.vertexDirectionalCurvatures[regionFront,2]/self.mesh.vertexCurvatures[regionFront]
        
        self.vertexMetrics.append("curvFront")        
        self.vertexMetrics.append("curvXFront")
        self.vertexMetrics.append("curvYFront")
        self.vertexMetrics.append("curvZFront")
        self.vertexMetrics.append("curvXPercentFront")
        self.vertexMetrics.append("curvYPercentFront")
        self.vertexMetrics.append("curvZPercentFront")

        
        regionBack=self.backRegionVertices()
        
        self.vertexResults["curvBack"]=self.mesh.vertexCurvatures[regionBack]
        self.vertexResults["curvXBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,0]
        self.vertexResults["curvYBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,1]
        self.vertexResults["curvZBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,2]
        self.vertexResults["curvXPercentBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,0]/self.mesh.vertexCurvatures[regionBack]
        self.vertexResults["curvYPercentBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,1]/self.mesh.vertexCurvatures[regionBack]
        self.vertexResults["curvZPercentBack"]=self.mesh.vertexDirectionalCurvatures[regionBack,2]/self.mesh.vertexCurvatures[regionBack]
        
        self.vertexMetrics.append("curvBack")        
        self.vertexMetrics.append("curvXBack")
        self.vertexMetrics.append("curvYBack")
        self.vertexMetrics.append("curvZBack")
        self.vertexMetrics.append("curvXPercentBack")
        self.vertexMetrics.append("curvYPercentBack")
        self.vertexMetrics.append("curvZPercentBack")
   

        for currMetric in self.vertexMetrics:
            self.vertexMeans[currMetric]=np.mean(self.vertexResults[currMetric])
            self.vertexSTDs[currMetric]=np.std(self.vertexResults[currMetric])
            
            
            

        self.globalResults["curvRatio"]=self.vertexMeans["curvFront"]/self.vertexMeans["curvBack"]
        self.globalResults["curvXRatio"]=self.vertexMeans["curvXFront"]/self.vertexMeans["curvXBack"]
        self.globalResults["curvYRatio"]=self.vertexMeans["curvYFront"]/self.vertexMeans["curvYBack"]
        self.globalResults["curvZRatio"]=self.vertexMeans["curvZFront"]/self.vertexMeans["curvZBack"]
        self.globalResults["curvXPercentRatio"]=self.vertexMeans["curvXPercentFront"]/self.vertexMeans["curvXPercentBack"]
        self.globalResults["curvYPercentRatio"]=self.vertexMeans["curvYPercentFront"]/self.vertexMeans["curvYPercentBack"]
        self.globalResults["curvZPercentRatio"]=self.vertexMeans["curvZPercentFront"]/self.vertexMeans["curvZPercentBack"]
        
        self.globalMetrics.append("curvRatio")        
        self.globalMetrics.append("curvXRatio")
        self.globalMetrics.append("curvYRatio")
        self.globalMetrics.append("curvZRatio")
        self.globalMetrics.append("curvXPercentRatio")
        self.globalMetrics.append("curvYPercentRatio")
        self.globalMetrics.append("curvZPercentRatio")


            
        self.globalResults["totalArea"] = np.sum(self.mesh.faceAreas[:])
        
        self.globalResults["regionAreaFront"]=0.0
        for v in np.nditer(np.nonzero(regionFront)):
            self.globalResults["regionAreaFront"]+=(np.sum(self.mesh.faceAreas[self.mesh.faceNeighbors[v]]))
        self.globalResults["regionAreaFrontPercentage"] = self.globalResults["regionAreaFront"] / self.globalResults["totalArea"]
        self.globalResults["regionAreaBack"]=0.0
        for v in np.nditer(np.nonzero(regionBack)):
            self.globalResults["regionAreaBack"]+=(np.sum(self.mesh.faceAreas[self.mesh.faceNeighbors[v]]))
        self.globalResults["regionAreaBackPercentage"] = self.globalResults["regionAreaBack"] / self.globalResults["totalArea"]
        
        self.globalMetrics.append("totalArea")        
        self.globalMetrics.append("regionAreaFront")
        self.globalMetrics.append("regionAreaFrontPercentage")
        self.globalMetrics.append("regionAreaBack")
        self.globalMetrics.append("regionAreaBackPercentage")
        

    def frontRegionVertices(self):
        #Cropping test based on geometric landmarks and normal vector angles (azimuth and elevation)

        #Crop mesh below the plane of the brow ridge and ear tips
        # to reduce noise from face and hair
        brp=self.mesh.vertices[self.mesh.browRidgeIndex]
        rep=self.mesh.vertices[self.mesh.rightEarIndex]
        lep=self.mesh.vertices[self.mesh.leftEarIndex]
        planeNormal=np.cross(rep-brp,lep-brp)
        abovePlane=np.sum((self.mesh.vertices-brp)*planeNormal,axis=1)<-5.0

        earY=(rep[1]+lep[1])/2
        inFront=self.mesh.vertices[:,1]>earY
        
        ##DYLAN FOR NOW USE ENTIRE PLANE, NOT JUST ANGLES
        finalCropIndices=np.logical_and(abovePlane,inFront)

        return finalCropIndices
        
    def backRegionVertices(self):
        #Cropping test based on geometric landmarks and normal vector angles (azimuth and elevation)

        #Crop mesh below the plane of the brow ridge and ear tips
        # to reduce noise from face and hair
        brp=self.mesh.vertices[self.mesh.browRidgeIndex]
        rep=self.mesh.vertices[self.mesh.rightEarIndex]
        lep=self.mesh.vertices[self.mesh.leftEarIndex]
        planeNormal=np.cross(rep-brp,lep-brp)
        abovePlane=np.sum((self.mesh.vertices-brp)*planeNormal,axis=1)<-5.0

        earY=(rep[1]+lep[1])/2
        inBack=self.mesh.vertices[:,1]<earY
        
        ##DYLAN FOR NOW USE ENTIRE PLANE, NOT JUST ANGLES
        finalCropIndices=np.logical_and(abovePlane,inBack)

        return finalCropIndices


In [47]:
class GroupResults:
    def __init__(self,resultList):
        self.resultList=resultList
        self.resultCount=len(resultList)
        
        self.results = dict()
        self.means = dict()
        self.stds = dict()

        self.metrics = list()
        
        self.metrics.append("curvFront")
        self.metrics.append("curvXFront")
        self.metrics.append("curvYFront")
        self.metrics.append("curvZFront")
        self.metrics.append("curvXPercentFront")
        self.metrics.append("curvYPercentFront")
        self.metrics.append("curvZPercentFront")
        self.metrics.append("curvBack")
        self.metrics.append("curvXBack")
        self.metrics.append("curvYBack")
        self.metrics.append("curvZBack")
        self.metrics.append("curvXPercentBack")
        self.metrics.append("curvYPercentBack")
        self.metrics.append("curvZPercentBack")
        
        for currMetric in self.metrics:
            self.results[currMetric]=np.empty(self.resultCount)
        
        for r in range(0,self.resultCount):
            for currMetric in self.metrics:
                (self.results[currMetric])[r]=(resultList[r]).vertexMeans[currMetric]        
        
        for currMetric in self.metrics:
            self.means[currMetric]=np.mean(self.results[currMetric])
            self.stds[currMetric]=np.std(self.results[currMetric])

    
  

In [20]:
import scipy.stats as stats

class GroupCompare:
    def __init__(self,groupResults1,groupResults2,metric):
        self.metric = metric
        n1=groupResults1.resultCount
        n2=groupResults2.resultCount
        s1=groupResults1.stds[self.metric]
        s2=groupResults2.stds[self.metric]
        x1=groupResults1.means[self.metric]
        x2=groupResults2.means[self.metric]
        
        sqdn1 = (s1**2)/n1
        sqdn2 = (s2**2)/n2
        
        self.se = np.sqrt( sqdn1 + sqdn2 )
        self.df = ((sqdn1+sqdn2)**2) / (((sqdn1**2)/(n1-1))+((sqdn2**2)/(n2-1)))
        self.t = np.abs((x1-x2)/self.se)
        self.p = stats.t.sf(self.t,self.df) * 2.0

END  - Classes and Functions

BEGIN  - Main Code

In [30]:
#Example on single medical scan

import os
import re
patientDir = '/home/dstow/research/med/scans/SickKids/Phillips/Ps0399_ESC'
dirlist = os.listdir(patientDir)
#patients = list()

#For single patientDir, saved pre and post ops
opList = os.listdir(patientDir)
for op in opList:
    opDir = patientDir+'/'+op
    if(re.search("^Pre",op)):
        fileList = os.listdir(opDir)
        simpFound=False
        for file in fileList:
            if(re.search("simp.obj",file)):
                simpFound=True
                fileName = opDir+'/'+file
                preOpObjFile = open(fileName,"rt")
        if(not simpFound):
            print("Error: *simp.obj file not found for directory "+opDir)
    match=re.search("^Post.*(\d+\.\d+)",op)
    if(match):
        postOpYears=match.group(1)
        fileList = os.listdir(opDir)
        simpFound=False
        for file in fileList:
            if(re.search("simp.obj",file)):
                simpFound=True
                fileName = opDir+'/'+file
                postOpObjFile = open(fileName,"rt")
        if(not simpFound):
            print("Error: *simp.obj file not found for directory "+opDir)
            
preOpMesh = obj_data_to_Mesh(preOpObjFile.read())
postOpMesh = obj_data_to_Mesh(postOpObjFile.read())

currPatient=Patient(preOpMesh,postOpMesh,"test",0,"test",0)

currResultPre=Results(currPatient,isPre=True)
currResultPost=Results(currPatient,isPre=False)

preOpRegionFront=currResultPre.frontRegionVertices()
postOpRegionFront=currResultPost.frontRegionVertices()
preOpRegionBack=currResultPre.backRegionVertices()
postOpRegionBack=currResultPost.backRegionVertices()


print(preOpMesh.vertexDirectionalCurvatures[preOpRegionFront,0])
print(preOpMesh.vertexCurvatures[preOpRegionFront])
print(preOpMesh.vertexDirectionalCurvatures[preOpRegionFront,0]/preOpMesh.vertexCurvatures[preOpRegionFront])
print(np.mean(preOpMesh.vertexDirectionalCurvatures[preOpRegionFront,0]/preOpMesh.vertexCurvatures[preOpRegionFront]))
print(preOpMesh.vertexDirectionalCurvatures[preOpRegionFront,0]+preOpMesh.vertexDirectionalCurvatures[preOpRegionFront,1]+preOpMesh.vertexDirectionalCurvatures[preOpRegionFront,2])


print("PreReg:", len(preOpRegionFront),"-", preOpRegionFront)
print("PostReg:", len(postOpRegionFront),"-", postOpRegionFront)

# - display landmarks for pre and post
preBrow = preOpMesh.browRidgeIndex
preREar = preOpMesh.rightEarIndex
preLEar = preOpMesh.leftEarIndex
vertexColors=np.ones_like(preOpMesh.vertices)
vertexColors[preBrow,0]=0.0
vertexColors[preREar,0]=0.0
vertexColors[preLEar,0]=0.0
vertexColors[np.nonzero(preOpRegionFront),1]=0.0
vertexColors[np.nonzero(preOpRegionBack),2]=0.0
plot_mesh_custom(preOpMesh,vertexColors,"Region Marker Debug PreOp")
postBrow = postOpMesh.browRidgeIndex
postREar = postOpMesh.rightEarIndex
postLEar = postOpMesh.leftEarIndex
vertexColors=np.ones_like(postOpMesh.vertices)
vertexColors[postBrow,0]=0.0
vertexColors[postREar,0]=0.0
vertexColors[postLEar,0]=0.0
vertexColors[np.nonzero(postOpRegionFront),1]=0.0
vertexColors[np.nonzero(postOpRegionBack),2]=0.0
plot_mesh_custom(postOpMesh,vertexColors,"Region Marker Debug PostOp")

print("Visualized patient file "+patientDir)

print(currResultPre.globalResults["curvXRatio"])
print(currResultPost.globalResults["curvXRatio"])

[0.00101412 0.00038809 0.00605326 ... 0.01296717 0.00397368 0.00560928]
[0.01738224 0.00933368 0.04121376 ... 0.02009792 0.01035063 0.00808699]
[0.05834214 0.04158003 0.14687481 ... 0.64519951 0.38390666 0.69361779]
0.3446585087173124
[0.02337336 0.01057446 0.03750514 ... 0.02619754 0.01467962 0.01314904]
PreReg: 4169 - [False False False ... False False False]
PostReg: 8608 - [False False False ... False False False]


Visualized patient file /home/dstow/research/med/scans/SickKids/Phillips/Ps0399_ESC
1.2033815446289016
0.890504010278101


In [38]:
import os
import re
startingDir = '/home/dstow/research/med/scans/SickKids'
dirlist = os.listdir(startingDir)
patientList = list()
for group in dirlist:
    groupDir = startingDir+'/'+group
    patientDirList = os.listdir(groupDir)
    for patient in patientDirList:
        patientDir = groupDir+'/'+patient
        preOpObjFile=""
        postOpObjFile=""
        patientNumber=""
        operationType=""
        postOpYear=999
        match=re.search("(\d+)_(\D+)$",patient)
        if(match):
            patientNumber=match.group(1)
            operationType=match.group(2)            
        else:
            print("Error: patient number or operation type not found for "+patient)
            break
        opDirList = os.listdir(patientDir)
        for op in opDirList:
            opDir = patientDir+'/'+op
            if(re.search("^Pre",op)):
                fileDirList = os.listdir(opDir)
                simpFound=False
                for file in fileDirList:
                    if(re.search("simp.obj",file)):
                        simpFound=True
                        fileName = opDir+'/'+file
                        preOpObjFile = open(fileName,"rt")
                if(not simpFound):
                    print("Error: *simp.obj file not found for directory "+opDir)
            match=re.search("^Post.*(\d+\.\d+)",op)
            if(match):
                postOpYears=match.group(1)
                fileDirList = os.listdir(opDir)
                simpFound=False
                for file in fileDirList:
                    if(re.search("simp.obj",file)):
                        simpFound=True
                        fileName = opDir+'/'+file
                        postOpObjFile = open(fileName,"rt")
                if(not simpFound):
                    print("Error: *simp.obj file not found for directory "+opDir)
        print("Creating meshes for patient file "+patientDir) 
        preOpMesh = obj_data_to_Mesh(preOpObjFile.read())
        postOpMesh = obj_data_to_Mesh(postOpObjFile.read())
        currPatient=Patient(preOpMesh,postOpMesh,group,patientNumber,operationType,postOpYears)
        patientList.append(currPatient)
        
        #print("Added result for patient file "+patientDir)
        #break
print("patientList count: ",len(patientList))

Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps2000_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1737_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1849_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1813_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1831_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1804_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/DrakeKulkarni/Ps1766_Endoscopic
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/Phillips/Ps1760_TCVR
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/Phillips/Ps1388_ESC
Creating meshes for patient file /home/dstow/research/med/scans/SickKids/Phill

In [39]:
resultListPre = list()
resultListPost = list()
for currPatient in patientList:
    currResultPre = Results(currPatient,isPre=True)
    currResultPost = Results(currPatient,isPre=False)
    resultListPre.append(currResultPre)
    resultListPost.append(currResultPost)
    
print("resultListPre count: ",len(resultListPre))
print("resultListPost count: ",len(resultListPost))

resultListPre count:  44
resultListPost count:  44


In [40]:
testPatient = patientList[16]
xcurv = testPatient.preOpMesh.vertexDirectionalCurvatures[17,0]
ycurv = testPatient.preOpMesh.vertexDirectionalCurvatures[17,1]
zcurv = testPatient.preOpMesh.vertexDirectionalCurvatures[17,2]
curv = testPatient.preOpMesh.vertexCurvatures[17]
print(curv,",",xcurv,",",ycurv,",",zcurv,",sum:",xcurv+ycurv+zcurv)

0.01425346549724209 , 0.006705568046530847 , 0.004568187895155791 , 0.002767156131157902 ,sum: 0.01404091207284454


In [89]:
endoResultsPre=list()
escResultsPre=list()
tcvrResultsPre=list()
endoResultsPost=list()
escResultsPost=list()
tcvrResultsPost=list()

print(len(resultListPre))

significance = 0.01


for currResultPre in resultListPre:
    if(currResultPre.operationType=="Endoscopic"):
        endoResultsPre.append(currResultPre)
    elif(currResultPre.operationType=="ESC"):
        escResultsPre.append(currResultPre)
    elif(currResultPre.operationType=="TCVR"):
        tcvrResultsPre.append(currResultPre)
    else:
        print("ERROR: no valid operation type for patient ")

endoGroupResultsPre=GroupResults(endoResultsPre)
escGroupResultsPre=GroupResults(escResultsPre)
tcvrGroupResultsPre=GroupResults(tcvrResultsPre)

#print("ESCPre: ",escGroupResultPre.resultCount)
#print("TCVRPre: ",tcvrGroupResultPre.resultCount)

print(len(resultListPost))

for currResultPost in resultListPost:
    if(currResultPost.operationType=="Endoscopic"):
        endoResultsPost.append(currResultPost)
    elif(currResultPost.operationType=="ESC"):
        escResultsPost.append(currResultPost)
    elif(currResultPost.operationType=="TCVR"):
        tcvrResultsPost.append(currResultPost)
    else:
        print("ERROR: no valid operation type for patient ")

endoGroupResultsPost=GroupResults(endoResultsPost)
escGroupResultsPost=GroupResults(escResultsPost)
tcvrGroupResultsPost=GroupResults(tcvrResultsPost)

print("Endo-ESC")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(endoGroupResultsPost,escGroupResultsPost,currMetric)
    print("Metric:",gc.metric," p:",gc.p," Diff:",gc.p<significance)

    
print("Endo-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(endoGroupResultsPost,tcvrGroupResultsPost,currMetric)
    print("Metric:",gc.metric," p:",gc.p," Diff:",gc.p<significance)
    
print("ESC-TCVR")
for currMetric in endoGroupResultsPost.metrics:
    gc=GroupCompare(escGroupResultsPost,tcvrGroupResultsPost,currMetric)
    print("Metric:",gc.metric," p:",gc.p," Diff:",gc.p<significance)

44
44
Endo-ESC
Metric: curvFront  p: 0.013882648658091103  Diff: False
Metric: curvXFront  p: 0.008060090002991517  Diff: True
Metric: curvYFront  p: 0.021790881824875968  Diff: False
Metric: curvZFront  p: 0.022191935055578385  Diff: False
Metric: curvXPercentFront  p: 0.02620897607996482  Diff: False
Metric: curvYPercentFront  p: 0.061177264613065406  Diff: False
Metric: curvZPercentFront  p: 0.6657627416553396  Diff: False
Metric: curvBack  p: 0.9536902947669841  Diff: False
Metric: curvXBack  p: 0.6137586108687899  Diff: False
Metric: curvYBack  p: 0.9252615945182309  Diff: False
Metric: curvZBack  p: 0.7256762474869085  Diff: False
Metric: curvXPercentBack  p: 0.9771618691035369  Diff: False
Metric: curvYPercentBack  p: 0.18203379216418444  Diff: False
Metric: curvZPercentBack  p: 0.008167409272637911  Diff: True
Endo-TCVR
Metric: curvFront  p: 0.12520555193308064  Diff: False
Metric: curvXFront  p: 0.038641496339280816  Diff: False
Metric: curvYFront  p: 0.24781184435615985  Diff

In [21]:
print(len(resultListPre))
print(len(resultListPost))

significance = 0.05

groupResultsPre=GroupResults(resultListPre)
groupResultsPost=GroupResults(resultListPost)

for currMetric in groupResultsPre.metrics:
    gc=GroupCompare(groupResultsPre,groupResultsPost,currMetric)
    print("Metric:",gc.metric," p:",gc.p," Diff:",gc.p<significance)
    
    
fig=go.Figure()
fig.add_trace(go.Scatter(x=groupResultsPre.results["curvYPercentBack"],y=groupResultsPre.results["curvZPercentFront"],mode='markers',name='Pre'))
fig.add_trace(go.Scatter(x=groupResultsPost.results["curvYPercentBack"],y=groupResultsPost.results["curvZPercentFront"],mode='markers',name='Post'))
fig.show()

44
44
Metric: curvFront  p: 0.1312407268106105  Diff: False
Metric: curvXFront  p: 0.3912365835486379  Diff: False
Metric: curvYFront  p: 0.009853168161566044  Diff: True
Metric: curvZFront  p: 0.44572930296031466  Diff: False
Metric: curvXPercentFront  p: 0.4143703696221708  Diff: False
Metric: curvYPercentFront  p: 0.06552886415362262  Diff: False
Metric: curvZPercentFront  p: 0.0006448756896514284  Diff: True
Metric: curvBack  p: 9.951597663353359e-05  Diff: True
Metric: curvXBack  p: 0.000594080993213901  Diff: True
Metric: curvYBack  p: 1.0471673026620615e-05  Diff: True
Metric: curvZBack  p: 0.00011056940225362593  Diff: True
Metric: curvXPercentBack  p: 0.04047092816041541  Diff: True
Metric: curvYPercentBack  p: 3.0232294964463774e-06  Diff: True
Metric: curvZPercentBack  p: 0.15533205446804865  Diff: False


In [ ]:
#Exploration - examine key metrics between pre and post

#resultCount = len(resultList)

#preMetrics = np.empty([resultCount,2])
#postMetrics = np.empty([resultCount,2])
#postMeshList = list()

#for i in range(0,resultCount):
#    preMetrics[i,0]=resultList[i].preMeanCurv
#    preMetrics[i,1]=resultList[i].postOpYears
    
#    postMetrics[i,0]=resultList[i].postMeanCurv
#    postMetrics[i,1]=resultList[i].postOpYears
    
#fig=go.Figure()
#fig.add_trace(go.Scatter(x=preMetrics[:,0],y=preMetrics[:,1],mode='markers',name='Pre'))
#fig.add_trace(go.Scatter(x=postMetrics[:,0],y=postMetrics[:,1],mode='markers',name='Post'))
#fig.show()

In [ ]:
if(False):

    #Extract data from 3D mesh vertices

    testExamples
    #Load file and check
    init_notebook_mode(connected=True)
    #objFileName = "test_robyn_simp.obj"
    objFileName = "test_simp.obj"
    objFile = open(objFileName,"rt")
    testmesh = obj_data_to_Mesh(objFile.read())
    print("Vertices: ",testmesh.vertices.shape)
    print("Faces: ",testmesh.faces.shape)


In [ ]:
if(False):

    plot_hist(np.log(testmesh.vertexDirectionalCurvatures[:,0])+0.0000000005,"Log(X-Curvature)")
    plot_hist(np.log(testmesh.vertexDirectionalCurvatures[:,1])+0.0000000005,"Log(Y-Curvature)")
    plot_hist(np.log(testmesh.vertexDirectionalCurvatures[:,2])+0.0000000005,"Log(Z-Curvature)")

    #Plot with color based on vertex position
    #Prepare color values based on x/y/z coordinate
    vertexColors=np.ones_like(testmesh.vertices)
    np.copyto(vertexColors,testmesh.vertices)
    vertexColors[:,0]-=testmesh.minX
    vertexColors[:,0]/=testmesh.rangeX
    vertexColors[:,1]-=testmesh.minY
    vertexColors[:,1]/=testmesh.rangeY
    vertexColors[:,2]-=testmesh.minZ
    vertexColors[:,2]/=testmesh.rangeZ

    plot_mesh_custom(testmesh,vertexColors,"Vertex Position")
    #Plot with color based on normal vector direction
    #Add one and divide by two to map from [-1,1] to [0,1]
    vertexColors=(testmesh.vertexNormals+1.0)/2.0
    plot_mesh_custom(testmesh,vertexColors,"Vertex Normals")
    plot_mesh_colorscale(testmesh,testmesh.vertexElevation,"Vertex Elevation","Elevation (Degrees)")
    plot_mesh_colorscale(testmesh,testmesh.vertexAzimuth,"Vertex Azimuth","Azimuth (Degrees)")
    #Plot with color based on curvature magnitude, log scale
    plot_mesh_colorscale(testmesh,np.log(testmesh.vertexCurvatures),"Vertex Curvature (Log)","Log(Curvature)")

    #Plot with color based on each direction curvature magnitude, log scale

    plot_mesh_colorscale(testmesh,np.log(testmesh.vertexDirectionalCurvatures[:,0]),"Vertex X-Curvature (Log)","Log(X-Curvature)")
    plot_mesh_colorscale(testmesh,np.log(testmesh.vertexDirectionalCurvatures[:,1]),"Vertex Y-Curvature (Log)","Log(Y-Curvature)")
    plot_mesh_colorscale(testmesh,np.log(testmesh.vertexDirectionalCurvatures[:,2]),"Vertex Z-Curvature (Log)","Log(Z-Curvature)")

Last Updated: 9/2/19